## Cleansing manual

output = ../data/cleaned/all_data_cleaned.pkl

In [1]:
import os
import re
import glob
import pickle
from collections import Counter

import pandas as pd
import numpy as np
import seaborn as sns

from nltk.tokenize import word_tokenize

%matplotlib inline

In [2]:
d_train_a = pd.read_csv('../data/raw/data-a/data_train_A.csv')

d_train_b = pd.read_csv('../data/raw/data-b/data_train_B.csv')

d_dev_a = pd.read_csv('../data/raw/data-a/data_dev_A.csv')

d_dev_b = pd.read_csv('../data/raw/data-b/data_dev_B.csv')

In [3]:
d_corrector = pd.read_csv('../data/support/word_freq_lessthan.csv', delimiter = ';', header = None)

d_corrector.columns = ["messy", "freq", "corrected"]

d_corrector = d_corrector[["messy", "corrected"]]

In [4]:
corrector = {}
for index in d_corrector.index:
    corrector[d_corrector.loc[index, 'messy']] = d_corrector.loc[index, 'corrected']

In [5]:
d_train_a['RESPONSE'] = d_train_a.RESPONSE.str.lower()
d_train_b['RESPONSE'] = d_train_b.RESPONSE.str.lower()

d_dev_a['RESPONSE'] = d_dev_a.RESPONSE.str.lower()
d_dev_b['RESPONSE'] = d_dev_b.RESPONSE.str.lower()

In [6]:
sentence_reconstruct = [
    ["TRA133", "harus dapat beradaptasi sama lingkungan sekitar wilayah baru"],
    ["TRA229", "karena keluarga travina mengungsi ke dataran rendah walaupun mengungsi akan kejadian banjir"],
    ["TRA19", "karena uangnya pun lebih bermanfaat untuk membeli kebutuhan hidup dibandingkan menyumbang pakaian"],
    ["TRA197", "penduduk meninggalkan perkebunan mereka ke daerah yang lebih tinggi untuk mengungsi"],
    ["TRA127", "menyebabkan kondisi kerja"],
    ["TRA197", "penduduk menigalkan perkebunan mereka ke daerah yang lebih tinggi untuk mengungsi"],
    ["TRA204", "akan sulit beradaptasi dengan lingkungan baru"],
    
    ["TRB202", "kenapa kita harus menyumbang karena kita sebagai manusia kita harus saling tolong menolong dan \
    untuk kebaikan juga sangat penting bagi manusia"],
    ["TRB201", "memastikan bahwa pekerja mendapatkan upah layak"],
    ["TRB129", "karena menyumbang itu berpahala"],
    ["TRB231", "menyebabkan kondisi kerja"],
    ["TRB159", "karena menyumbang itu berpahala"],
    
    ["DEVA3", "susah berkomunikasi dengan wilayah yang mereka tempati ."],
    
    
    ["DEVB71", "karena konsumen sudah tahu bahwa kondisi kerja yang buruk di pabrik pakaian, \
    akan dibuang ke tempat pembuangan sampah. maka dari itu konsumen lebih memilih menyumbangkan \
    uang sebesar 2 euro (atau sekitar rp. 30.000) dibandingkan dengan membeli baju yg kualitasnya tidak begitu bagus"],
]

In [7]:
len(sentence_reconstruct)

14

In [8]:
for sentence in sentence_reconstruct:
    train_a_index = d_train_a[d_train_a["RES_ID"] == sentence[0]].index
    train_b_index = d_train_b[d_train_b["RES_ID"] == sentence[0]].index
    dev_a_index = d_dev_a[d_dev_a["RES_ID"] == sentence[0]].index
    dev_b_index = d_dev_b[d_dev_b["RES_ID"] == sentence[0]].index
    
    if len(train_a_index):
        d_train_a.loc[train_a_index, 'RESPONSE'] = sentence[1]
        print(sentence[0])
    elif len(train_b_index):
        d_train_b.loc[train_b_index, "RESPONSE"] = sentence[1]
        print(sentence[0])
    elif len(dev_a_index): 
        d_dev_a.loc[dev_a_index, "RESPONSE"] = sentence[1]
        print(sentence[0])
    elif len(dev_b_index):
        d_dev_b.loc[dev_b_index, "RESPONSE"] = sentence[1]
        print(sentence[0])
    else:
        print("not found", sentence[0])

TRA133
TRA229
TRA19
TRA197
TRA127
TRA197
TRA204
TRB202
TRB201
TRB129
TRB231
TRB159
DEVA3
DEVB71


In [9]:
def rep_corrector(word_list):
    word_corrected = []
    for word in word_list:
        word = str(word)
        try:
            word = corrector[word]
        except:
            pass
        
        word_corrected.append(word)

    return " ".join(word_corrected)

In [10]:
d_train_a["word_list"] = d_train_a.RESPONSE.apply(word_tokenize)
d_train_b["word_list"] = d_train_b.RESPONSE.apply(word_tokenize)
d_dev_a["word_list"] = d_dev_a.RESPONSE.apply(word_tokenize)
d_dev_b["word_list"] = d_dev_b.RESPONSE.apply(word_tokenize)

In [11]:
for index in d_train_a.index:
    d_train_a.loc[index, "response_2"] = rep_corrector(d_train_a.loc[index, "word_list"])

In [12]:
for index in d_train_b.index:
    d_train_b.loc[index, "response_2"] = rep_corrector(d_train_b.loc[index, "word_list"])

In [13]:
for index in d_dev_a.index:
    d_dev_a.loc[index, "response_2"] = rep_corrector(d_dev_a.loc[index, "word_list"])

In [14]:
for index in d_dev_b.index:
    d_dev_b.loc[index, "response_2"] = rep_corrector(d_dev_b.loc[index, "word_list"])

In [15]:
data_dict = {
    'train_a': d_train_a,
    'train_b': d_train_b,
    'dev_a': d_dev_a,
    'dev_b': d_dev_b
}

In [16]:
d_dev_a

,RES_ID,RESPONSE,word_list,response_2
0,DEVA1,karena di tempat tnggalnya tidak aman karena b...,"[karena, di, tempat, tnggalnya, tidak, aman, k...",karena di tempat tinggalnya tidak aman karena ...
1,DEVA2,"banyak yang menolak, susah mencari lapangan pe...","[banyak, yang, menolak, ,, susah, mencari, lap...","banyak yang menolak , susah mencari lapangan p..."
2,DEVA3,susah berkomunikasi dengan wilayah yang mereka...,"[susah, berkomunikasi, dengan, wilayah, yang, ...",susah berkomunikasi dengan wilayah yang mereka...
3,DEVA4,"tantangan yang akan mereka hadapi, seperti mer...","[tantangan, yang, akan, mereka, hadapi, ,, sep...","tantangan yang akan mereka hadapi , seperti me..."
4,DEVA5,mereka akan tidak terbiasa dengan tempat baru,"[mereka, akan, tidak, terbiasa, dengan, tempat...",mereka akan tidak terbiasa dengan tempat baru
5,DEVA6,kehilangan pekerjaan dan sumber daya yang ada ...,"[kehilangan, pekerjaan, dan, sumber, daya, yan...",kehilangan pekerjaan dan sumber daya yang ada ...
6,DEVA7,dengan sikap yang tenang,"[dengan, sikap, yang, tenang]",dengan sikap yang tenang
7,DEVA8,mencari tempat pengungsian,"[mencari, tempat, pengungsian]",mencari tempat pengungsian
8,DEVA9,cuaca/atau suasana di perjalanan,"[cuaca/atau, suasana, di, perjalanan]",cuaca / atau suasana di perjalanan
9,DEVA10,pengungsi iklim lingkungan baru yang berbeda d...,"[pengungsi, iklim, lingkungan, baru, yang, ber...",pengungsi iklim lingkungan baru yang berbeda d...


In [ ]:
pickle.dump(data_dict, open("../data/cleaned/all_data_cleaned.pkl", 'wb'))